- In this assignment we will be creating a multi-layer network that will be able to learn what a healthy lung looks like vs a non healthy lung that has Pnuemonia. 

- first step: PARSE THE DATA VERYYYYY TIME CONSUMMING


In [1]:
from PIL import Image
#from resizeimage import resizeimage
import numpy as np
import tensorflow as tf
import tensorflow.contrib.eager as tfe
from matplotlib import pyplot as plt
from os import listdir

tf.enable_eager_execution()

In [2]:
#data_set_url = "/Users/benjaminadame/Downloads/chest_xray"
data_set_url = "/home/gautam/.keras/datasets/chest_xray"

normal_url_train = data_set_url + "/train/NORMAL/"
pneumonia_url_train = data_set_url + "/train/PNEUMONIA/"
normal_url_test = data_set_url + "/test/NORMAL/"
pneumonia_url_test = data_set_url + "/test/PNEUMONIA/"
input_size = 2500

#initializing training and testing matrices 
def initialize_test_train(n,p):
    #size of the image that we want
    size = [50,50]
    i = 0
    #Open trainng directorys
    n_d = listdir(n)
    p_d = listdir(p)
        
    #nnp: number of normal pictures in training data
    nnp = len(n_d)
    #npp: number of pneumonia pictures in training data
    npp = len(p_d)
    #row is speaking of image size which is 50*50*3
    x = np.zeros((nnp+npp, input_size))
    #row is speaking about label 1 label for each
    y = np.zeros((nnp+npp,1))
    #Process NORMAL lung files for training and testing set 
    for file in n_d:
        #print(file)    
        #Resizing images
        with open(n + file, 'r+b') as f:
            with Image.open(f) as image:
                cover = image.resize((50,50))
                cover = np.array(cover)
                if cover.shape == (50, 50):
                    cover = cover.reshape(1,input_size)
                    x[i,:] = cover
                    y[i,0] = 0
                    i += 1
            
    #Process PNEUMONIA lung files for training and testing set 
    for file in p_d:
        #print(file)
        #Resize images
        with open(p + file, 'r+b') as f:
            with Image.open(f) as image:
                cover = image.resize((50,50))
                cover = np.array(cover)
                if cover.shape == (50,50):
                    cover = cover.reshape(1,input_size)
                    x[i, :] = cover
                    y[i,0] = 1
                    i += 1
                    
    return x[0:i,:], y[0:i,:]
    
#call x and y to initialize matrices
X_train, Y_train = initialize_test_train(normal_url_train,pneumonia_url_train)
X_test, Y_test = initialize_test_train(normal_url_test,pneumonia_url_test)

#print(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)


In [3]:
print(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)

(4949, 2500) (624, 2500) (4949, 1) (624, 1)


In [4]:
#Tensorflow Datasets
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
Y_train = tf.convert_to_tensor(Y_train, dtype=tf.float32)
Y_test = tf.convert_to_tensor(Y_test, dtype=tf.float32)

#create train dataset
train_dataset = tf.data.Dataset.from_tensors((X_train,Y_train))
test_dataset = tf.data.Dataset.from_tensors((X_test,Y_test))



Construct the Network Architechture

In [5]:
#model architechture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(5,activation = "relu",input_shape = (2500,)),
    tf.keras.layers.Dense(3,activation = "relu"),
    tf.keras.layers.Dense(2,activation = "relu"),
    tf.keras.layers.Dense(1,activation = "sigmoid")
])

Construct the foward propagation function 

In [6]:
def forward_prop(model,x,y):
    a = model(x)
    loss = tf.losses.sigmoid_cross_entropy(y,a)
    return loss

Get Gradienr Descent //optimizer 


In [7]:
lr = .1
optimizer = tf.train.GradientDescentOptimizer(learning_rate = lr)

Training Time!!!

In [11]:
m = 5232
epochs = 1000

#loop it
for epoch in range(epochs):
    loss_avg = tfe.metrics.Mean()
    
    for x,y in train_dataset:
        with tf.GradientTape(persistent = True) as tape:
            loss = forward_prop(model,x,y)
            #grad: gives you dw,db while model.vars gives you w and b
        grad = tape.gradient(loss,model.variables)
        loss_avg(loss)
        #update with grad descent 
        optimizer.apply_gradients(zip(grad,model.variables))
        
    if epoch % 10 == 0:
        print("cost: {}".format(loss_avg.result()))

cost: 0.6049038171768188
cost: 0.6044018268585205
cost: 0.6038941144943237
cost: 0.6034396290779114
cost: 0.6029759049415588
cost: 0.6025326251983643
cost: 0.6020899415016174
cost: 0.6016761064529419
cost: 0.6012721657752991
cost: 0.6009052395820618
cost: 0.6005217432975769
cost: 0.6001666188240051
cost: 0.5998056530952454
cost: 0.5994661450386047
cost: 0.599143385887146
cost: 0.5988265872001648
cost: 0.5985205173492432
cost: 0.5982186198234558
cost: 0.5979374051094055
cost: 0.5976781249046326
cost: 0.597405731678009
cost: 0.5971387028694153
cost: 0.5968919992446899
cost: 0.5966417789459229
cost: 0.5964164733886719
cost: 0.5962035655975342
cost: 0.5959715247154236
cost: 0.595751941204071
cost: 0.5955595374107361
cost: 0.5953627228736877
cost: 0.5951564311981201
cost: 0.5949842929840088
cost: 0.5948003530502319
cost: 0.5946063995361328
cost: 0.5944474339485168
cost: 0.5942801833152771
cost: 0.5941110253334045
cost: 0.59395831823349
cost: 0.5938080549240112
cost: 0.593654990196228
cost: 